Для начала импортируем необходимые библиотеки:

In [ ]:
import numpy as np #для матричных вычислений
import pandas as pd #для анализа и предобработки данных
import matplotlib.pyplot as plt #для визуализации
import seaborn as sns #для визуализации

from sklearn import metrics #метрики
from sklearn import model_selection #методы разделения и валидации
from sklearn import ensemble #ансамбли

plt.style.use('seaborn-v0_8') #стиль отрисовки seaborn
%matplotlib inline

1. Прочитайте таблицу с данными ('data/online_shoppers_intention.csv') и выведите ее на экран, чтобы убедиться, что чтение прошло успешно.

In [ ]:
df = pd.read_csv('data/online_shoppers_intention.csv')
print(df.head())

2. Выведите размер таблицы

In [ ]:
print(f"Строк {df.shape[0]}. Столбцов {df.shape[1]}")
assert df.shape[0] > 12000 and df.shape[1] == 18

3. В нашей таблице содержится информация о более чем 12 тысячах сессий на сайте интернет-магазина. Каждая сессия описывается 18 признаками.
Удостоверьтесь в отсутствии пропусков:

In [ ]:
nulls = df.isnull().sum()
print(nulls)
for col, sum in nulls.items():
    assert sum-=0, f"Имеются пропуски в колонке {col}"

4. Закодируйте категориальные признаки с помощью простого горячего кодирования, используя уже знакомую нам функцию get_dummies():

In [ ]:
types = df.dtypes
#Категориальные признаки
cat_features = list(types[(types == 'object')].index)
print(cat_features) # ['Month', 'VisitorType']

fig, axes = plt.subplots(1, 2, figsize=(40, 20))
#Строим столбчатую диаграмму для категории Месяц
sns.barplot(data=df, x=cat_features[0], y='Revenue', ax=axes[0])
#Строим столбчатую диаграмму для категории Тип посетителя
sns.barplot(data=df, x=cat_features[1], y='Revenue', ax=axes[1])

plt.tight_layout()
plt.show()
#по столбчатым диаграммам видим что максимум приходится на конец года - Октябрь, Ноябрь, Декабь
# в Январе и феврале - минимум продаж - этот признак отбрасывать нельзя

#по типу клиента связь выражена не так сильно как по месяцам, но все-же есть. (тоже не отбрасываем)

print(df.describe(include='object'))
df_dummies = pd.get_dummies(df)
print(f"Описание после преобразования категориальных признаков. Количество колонок {df_dummies.dtypes.shape}")
print(df_dummies.dtypes)

5. Теперь, когда необходимые преобразования выполнены, мы можем говорить о построении модели.

Итак, нам необходимо предсказать целевую переменную Revenue — признак покупки. Целевой признак является бинарным категориальным, то есть мы решаем задачу бинарной классификации.
В первую очередь визуализируйте соотношение классов в данных:

In [ ]:
sns.countplot(data=df, x='Revenue')
plt.show()

plt.figure(figsize=(20, 20))
corr = df_dummies.corr(method='spearman')
sns.heatmap(corr, annot=True, cmap='coolwarm')
plt.show()

# По heatmap видимо что признаки Administrative_Duration, Informational, Informational_Duration, ProductRelated, ProductRelated_Duration -
# коррелируют между собой плюс минус одинаково, что приведет нас к мультиколлинеарности при использовании линейной или пол. регрессии
# Будем использовать Случайный лес
# Суть этого метода заключается в том, что каждая модель обучается не на всех признаках, а только на части из них.
# Такой подход позволяет уменьшить коррелированность между ответами деревьев и сделать их независимыми друг от друга.

# Также видно что некоторые аттрибуты слабо коррелируют по отношению к целевому признаку
# Удаляем те, что ниже 0.02
df_dummies = df_dummies.drop(['OperatingSystems', 'Browser', 'TrafficType', 'Month_Aug', 'Month_Jul', 'VisitorType_Other'], axis=1)

6. Сбалансирована ли данная выборка? Обоснуйте свою позицию

In [ ]:
Выборка не сбалансирована. Купленных машин в несколько раз меньше по завершении сессии.

7. Из 12330 сессий покупкой товара завершаются лишь 15.47 %. Мы знаем, что такое соотношение классов заставляет нас смотреть на метрики для каждого из классов отдельно.

Условимся, что лучшей будет считаться та модель, у которой значение метрики F1 для пользователей, совершивших покупку, будет наибольшим.

Разделите набор данных на матрицу наблюдений X и вектор ответов y:

In [ ]:
X = df_dummies.drop('Revenue', axis=1)
y = df_dummies['Revenue']
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, random_state=42, test_size=0.2, shuffle=True)

december_train = X_train[X_train['Month_Dec'] == True].count()['Month_Dec']
december_test = X_test[X_test['Month_Dec'] == True].count()['Month_Dec']
#убеждаемся, что данные за декабрь попали как в тест, так и в трейн
print(f"Декабрьских сессий в трейне {december_train} и в тесте {december_test}")
#Декабрьских сессий в трейне 1387 и в тесте 340

#Соотношение целевого признака в train и в test (должно быть примерно одинаковым)
print('Train:\n', y_train.value_counts(normalize=True), sep='')
print('Valid:\n', y_test.value_counts(normalize=True), sep='')
#Revenue
#False    0.848236
#True     0.151764
#Name: proportion, dtype: float64
#Valid:
#Revenue
#False    0.833333
#True     0.166667
#Name: proportion, dtype: float64

Давайте заранее определимся, как мы будем производить контроль качества наших моделей:

Разделим выборку на тренировочную и тестовую.
Будем проводить кросс-валидацию на тренировочной выборке (то есть будем делить её на тренировочные и валидационные фолды и считать среднее значение метрики по фолдам).
Итого мы будем использовать два показателя:

значение метрики на тренировочных и валидационных фолдах кросс-валидации (по ним мы будем отслеживать переобучение модели и подбирать внешние параметры);
значение метрики на отложенной тестовой выборке (оно будет нашим контрольным показателем).
Другими словами, мы будем сочетать hold-оut- и k-fold-подходы к валидации.

8. Для начала позаботимся о создании отложенной тестовой выборки.

Разделите выборку на тренировочную и тестовую в соотношении 80/20. Используйте разбиение, стратифицированное по целевому признаку. В качестве значения параметра random_state возьмите число 42.

Чему равно количество сессий на сайте в тренировочной выборке?

In [ ]:
print(f"всего данных трейн {y_train.count()} тест {y_test.count()}")
#всего данных трейн 9864 тест 2466

9. Расчитайте количество сессий в тестовой выборке:

In [ ]:
2466

10.Коллеги посоветовали нам использовать случайный лес (Random Forest) для решения данной задачи. Давайте последуем их совету.

Создайте модель случайного леса. В качестве значения параметра random_state возьмите число 42. Остальные параметры оставьте по умолчанию.

Оцените качество такой модели с помощью кросс-валидации по пяти фолдам. Так как классы несбалансированы, используйте кросс-валидатор StratifiedKFold (перемешивать выборку не нужно).

Для проведения кросс-валидации используйте функцию cross_validate(). Набор данных (параметры X, y) — тренировочная выборка (X_train, y_train). Метрика — F1-score.

Расчитайте, чему равно среднее значение метрики  на тренировочных и валидационных фолдах?

In [ ]:
#Производим нормализацию данных с помощью min-max нормализации
scaler = preprocessing.MinMaxScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

#Создаем объект класса случайный лес
rf = ensemble.RandomForestClassifier(
    random_state=42
)
#Обучаем модель
rf.fit(X_train, y_train)
#Выводим значения метрики
y_train_pred = rf.predict(X_train)
print('Train: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
y_test_pred = rf.predict(X_test)
print('Test: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))


11. Является ли, по-вашему, построенная в предыдущем задании модель случайного леса переобученной? Обоснуйте вашу позицию

In [ ]:
Train: 1.00
Test: 0.62
Да, переобучено, так как на тренировочных данных идеальный результат, а на тестовых плохой.

12. Попробуем использовать несколько вариаций случайного леса и с помощью кривых обучения постараемся выбрать наилучшую из них.

Создайте список из трёх следующих моделей:

Случайный лес из деревьев максимальной глубины 5.
Случайный лес из деревьев максимальной глубины 7.
Случайный лес из деревьев максимальной глубины 12.
Для всех трёх моделей количество деревьев в лесу (n_estimators) возьмите равным 200, количество объектов в листе (min_samples_leaf) — 5. Параметр random_state = 42. Остальные параметры оставьте по умолчанию.

Постройте для каждой из моделей кривую обучения.
Совет: воспользуйтесь функцией plot_learning_curve()

Для построения кривых используйте обучающий набор данных (X_train, y_train), стратифицированный кросс-валидатор на пяти фолдах (StratifiedKFold) и метрику F1-score. Остальные параметры функции learning_curve() оставьте по умолчанию.

In [ ]:
def plot_learning_curve(model, X, y, ax=None, title=""):
    kf = model_selection.StratifiedKFold(n_splits=5)
    train_sizes, train_scores, valid_scores = model_selection.learning_curve(
        estimator=model,  # модель
        X=X,  # матрица наблюдений X
        y=y,  # вектор ответов y
        cv=kf,  # кросс-валидатор
        scoring='f1'  # метрика
    )
    # Вычисляем среднее значение по фолдам для каждого набора данных
    train_scores_mean = np.mean(train_scores, axis=1)
    valid_scores_mean = np.mean(valid_scores, axis=1)
    # Строим кривую обучения по метрикам на тренировочных фолдах
    ax.plot(train_sizes, train_scores_mean, label="Train")
    # Строим кривую обучения по метрикам на валидационных фолдах
    ax.plot(train_sizes, valid_scores_mean, label="Valid")
    # Даём название графику и подписи осям
    ax.set_title("Learning curve: {}".format(title))
    ax.set_xlabel("Train data size")
    ax.set_ylabel("Score")
    # Устанавливаем отметки по оси абсцисс
    ax.xaxis.set_ticks(train_sizes)
    # Устанавливаем диапазон оси ординат
    ax.set_ylim(0, 1)
    # Отображаем легенду
    ax.legend()

rf5 = ensemble.RandomForestClassifier(
    criterion='entropy',
    n_estimators=200,
    max_depth=5,
    min_samples_leaf=5,
    random_state=42
)
rf7 = ensemble.RandomForestClassifier(
    criterion='entropy',
    n_estimators=200,
    max_depth=7,
    min_samples_leaf=5,
    random_state=42
)
rf12 = ensemble.RandomForestClassifier(
    criterion='entropy',
    n_estimators=200,
    max_depth=12,
    min_samples_leaf=5,
    random_state=42
)

fig, axes = plt.subplots(1, 3, figsize=(15, 4)) #фигура + 3 координатных плоскости
plot_learning_curve(rf5, X_train_scaled, y_train, ax=axes[0], title='Глубина 5')
plot_learning_curve(rf7, X_train_scaled, y_train, ax=axes[1], title='Глубина 7')
plot_learning_curve(rf12, X_train_scaled, y_train, ax=axes[2], title='Глубина 12')
plt.show()

13. Из построенных кривых обучения сделайте вывод: какая глубина деревьев в лесу является оптимальной? Ответ обоснуйте

In [ ]:
f1 метрика у модели с глубиной в 12 выше чем у модели 7 и 5. Если не обращать внимание на затраты на вычисление, нужно брать её.

14. Обучите случайный лес с выбранной в предыдущем задании оптимальной глубиной на тренировочной выборке. Сделайте предсказание меток классов и выведите отчёт о метриках классификации.

 Рассчитайте значение метрики accuracy

In [ ]:
def random_forest_train(model, X, y, scoring):
    # Создаём объект кросс-валидатора KFold
    kf = model_selection.StratifiedKFold(n_splits=5)
    # Считаем метрики на кросс-валидации k-fold
    cv_metrics = model_selection.cross_validate(
        estimator=model,  # модель
        X=X,  # матрица наблюдений X
        y=y,  # вектор ответов y
        cv=kf,  # кросс-валидатор
        scoring=scoring,  # метрика
        return_train_score=True  # подсчёт метрики на тренировочных фолдах
    )
    #print(cv_metrics)
    print('Train k-fold mean accuracy: {:.2f}'.format(np.mean(cv_metrics['train_score'])))
    print('Valid k-fold mean accuracy: {:.2f}'.format(np.mean(cv_metrics['test_score'])))

random_forest_train(rf12, X_train_scaled, y_train, 'accuracy')    
#Train k-fold mean accuracy: 0.94
#Valid k-fold mean accuracy: 0.91

15. Рассчитайте значение метрики F1 для посетителей, завершивших сессию без покупки товара?

In [ ]:
df_no_revenue = df_dummies.copy()
df_no_revenue = df_no_revenue[df_no_revenue['Revenue'] == False]
print(f"Количество сессий без покупки {df_no_revenue.shape[0]}")
X_no_revenue = df_no_revenue.drop('Revenue', axis=1)
y_no_revenue = df_no_revenue['Revenue']
X_no_revenue_scaled = scaler.transform(X_no_revenue)
y_no_revenue_pred = rf12.predict(X_no_revenue_scaled)
print('F1 no revenue: {:.2f}'.format(metrics.f1_score(y_no_revenue, y_no_revenue_pred)))
#Количество сессий без покупки 10422
#F1 no revenue: 0.00

16. Рассчитайте значение метрики F1 для посетителей, купивших товар во время сессии?

In [ ]:
df_revenue = df_dummies.copy()
df_revenue = df_revenue[df_revenue['Revenue'] == True]
print(f"Количество сессий завершившихся покупкой {df_revenue.shape[0]}")
X_revenue = df_revenue.drop('Revenue', axis=1)
y_revenue = df_revenue['Revenue']
X_revenue_scaled = scaler.transform(X_revenue)
y_revenue_pred = rf12.predict(X_revenue_scaled)
print('F1 revenue: {:.2f}'.format(metrics.f1_score(y_revenue, y_revenue_pred)))
#Количество сессий завершившихся покупкой 1908
#F1 revenue: 0.78

17. Напишите вывод по полученным значениям

In [ ]:
Результат странный, по идее целевой признак бинарный и F1 должен был быть одинаков - т.е. если хорошо предсказываем один,
то автоматически предсказыаем и другой.

18. Попробуем повысить качество распознавания посетителей, совершивших покупку. Используем метод подбора порога вероятности с помощью PR-кривой.

Порог вероятности будем подбирать с помощью кросс-валидации.

Сделайте предсказание вероятностей принадлежности к пользователям, которые совершат покупку, на кросс-валидации на пяти фолдах. Используйте метод cross_val_predict().

Для кросс-валидации используйте случайный лес с подобранной в прошлых заданиях оптимальной максимальной глубиной деревьев, набор данных (параметры X, y) — тренировочная выборка (X_train, y_train).

Постройте PR-кривую и отметьте на ней точку, в которой наблюдается максимум метрики  для посетителей, которые совершат покупку. Определите порог вероятности, соответствующий этой точке.

19. Сделайте предсказание классов объекта с определённым в предыдущем задании порогом вероятности. Выведите отчёт о метриках классификации.
Рассчитайте значение метрики accuracy

20. Рассчитайте значение метрики F1 для посетителей, завершивших сессию без покупки товара и значение метрики F1 для посетителей, купивших товар во время сессии? После расчета напишите свои выводы на основе метрик